# This notebook will contain functions which i had found usefull

 ### I will be using my use case datasets to run all examples :

In [2]:
###################################################################
# Libraries used:
###################################################################
import numpy as np
import pandas as pd
import seaborn as sns
import folium
import matplotlib.pyplot as plt
import requests
import math
from geopy.distance import geodesic

In [3]:
# Manually reading each CSV file from website
linkLitterTraps = "https://data.melbourne.vic.gov.au/api/explore/v2.1/catalog/datasets/litter-traps/exports/csv?lang=en&timezone=Australia%2FSydney&use_labels=true&delimiter=%2C"
linkPublicBBQ ="https://data.melbourne.vic.gov.au/api/explore/v2.1/catalog/datasets/public-barbecues/exports/csv?lang=en&timezone=Australia%2FSydney&use_labels=true&delimiter=%2C"
linklinkCafeRes = "https://data.melbourne.vic.gov.au/api/explore/v2.1/catalog/datasets/cafes-and-restaurants-with-seating-capacity/exports/csv?lang=en&timezone=Australia%2FSydney&use_labels=true&delimiter=%2C"
litter_df = pd.read_csv(linkLitterTraps)
bbq_df = pd.read_csv(linkPublicBBQ)
cafe_df = pd.read_csv(linklinkCafeRes)

### remove NULL values first :

In [4]:
# Handle NaN values in the datasets ## TO BE CHECKED - Thanks Thomas
litter_df.dropna(subset=['lat', 'lon'], inplace=True)
bbq_df.dropna(subset=['Co-ordinates'], inplace=True)
cafe_df.dropna(subset=['Latitude', 'Longitude'], inplace=True)



## Extract coords

In [5]:
# Extract coordinates as tuples (latitude, longitude) as a list example :
"""
This rteally depends on how your dataset handdles the storing of the coordinates / location , sometimes you dont need to 
do this
"""

litter_coords = list(zip(litter_df['lat'], litter_df['lon'])) 

bbq_coords = [(float(c.split(',')[0]), float(c.split(',')[1])) for c in bbq_df['Co-ordinates']]

cafe_coords = list(zip(cafe_df['Latitude'], cafe_df['Longitude']))

## Point to point distance calculator minimum ( Thomas )


In [39]:
# Function to calculate the minimum distance from a point to any point in a list
"""
Calculate the minimum geodesic distance from a point to any point in a given list.

Parameters:
point (tuple): A tuple representing the coordinates (latitude, longitude) of the point.
list_of_points (list of tuples): A list of tuples, each representing coordinates (latitude, longitude) of points to compare against.

Returns:
float: The minimum Euclidean distance from the given point to the closest point in the list.
"""

def min_distance(point, list_of_points): 
    return min([geodesic(point, pt).meters for pt in list_of_points]) #get min dis

#example :


row = {'lat': 40.7128, 'lon': -74.0060}
# Call the lambda function with the row as an argument
value = lambda row: min_distance((row['lat'], row['lon']), bbq_coords)
# Get the result by calling the lambda function
result = value(row)
# Print the result
print("test distance in meters :",result)

# example used in dataset :


litter_df['Nearest BBQ Distance (m)'] = litter_df.apply(lambda row: min_distance((row['lat'], row['lon']), bbq_coords), axis=1)
#creates a new column for nearest distance to a point

test distance in meters : 16669936.382948814


## Point to point distance calculator maximum


In [7]:
# Function to calculate the maximum distance from a point to any point in a list
"""
Calculate the maximum geodesic distance from a point to any point in a given list.

Parameters:
point (tuple): A tuple representing the coordinates (latitude, longitude) of the point.
list_of_points (list of tuples): A list of tuples, each representing coordinates (latitude, longitude) of points to compare against.

Returns:
float: The maximum Euclidean distance from the given point to the closest point in the list.
"""

def max_distance(point, list_of_points): 
    return max([geodesic(point, pt).meters for pt in list_of_points]) #get min dis

#example : 

value = max_distance((row['lat'], row['lon']), bbq_coords) 

## Number of points in a given radius 

In [8]:
#Calculate the Number of points in a radius from a point 
"""
Calculate the number of geodesic distances from a point to any point in a given list.

Parameters:
center_point (tuple): A tuple representing the coordinates (latitude, longitude) of the point.
list_of_points (list of tuples): A list of tuples, each representing coordinates (latitude, longitude) of points to compare against.
radius_meters

Returns:
INT: The Number of points in the radius given
"""

def count_points_in_radius(center_point, list_of_points, radius_meters):
    count = sum(1 for pt in list_of_points if geodesic(center_point, pt).meters <= radius_meters)
    return count

#Example into dataset : 

#========Parameter 1 : Centur point 
#========Parameter 2 : all coordinate points [must be list form , see example ]
#========Parameter 3 : radius 

radius = 100
litter_df['Number of Nearby Points in Radius'] = litter_df.apply(lambda row: count_points_in_radius((row['lat'], row['lon']), bbq_coords + cafe_coords,radius),axis=1)

# Example ( singular ) :

values = count_points_in_radius((row['lat'], row['lon']),cafe_coords,radius)

## The Map using folium ( basic )


In [14]:
litter_df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 63 entries, 0 to 62
Data columns (total 11 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   asset_number                       63 non-null     int64  
 1   asset_description                  63 non-null     object 
 2   construct_material_lupvalue        56 non-null     object 
 3   inspection_frequency               58 non-null     object 
 4   maintained_by                      63 non-null     object 
 5   object_type_lupvalue               59 non-null     object 
 6   lat                                63 non-null     float64
 7   lon                                63 non-null     float64
 8   location                           63 non-null     object 
 9   Nearest BBQ Distance (m)           63 non-null     float64
 10  Number of Nearby Points in Radius  63 non-null     int64  
dtypes: float64(3), int64(2), object(6)
memory usage: 5.5+ KB


In [37]:
from folium.plugins import MarkerCluster

    
"""
Calculate the minimum geodesic distance from a point to any point in a given list.

Parameters:
dataframe : A datset representing the coordinates (latitude, longitude) of the index and also other values hence when
using this we can also include other things from the dataset in the map , when using the html legend

Returns:
Map: The folium based map is returned
"""

def map_func(PointsDatasets,):
    # Create a folium map centered at the mean coordinates of litter traps / intial setup
    map_center = [PointsDatasets['lat'].mean(), PointsDatasets['lon'].mean()]
    mymap = folium.Map(location=map_center, zoom_start=13)
    
    # Add circles for the points
    for index, row in PointsDatasets.iterrows():
        location = [row['lat'], row['lon']] 
        # Add a circle for the radius around the litter trap
        folium.Circle(
            location=location,
            radius=30,
            color='red',
            fill=True,
            fill_opacity=0.2
        ).add_to(mymap)
    return mymap

# Example usage ========================= Pass in your function =================
"""Make sure your dataframe has a column with both lat and lon"""
map_func(litter_df)